In [15]:
#1. NEJM germline mutation
#pmid 26580448
#The processing directory will be /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/vcf.germline
#changes comparing with old version: (1): change tumor_DNA to tumor_DNA_WGS/WES or both
#tumor_DNA_WGS, germline_DNA_WGS, tumor_DNA_WES, germline_DNA_WES, tumor_RNA



import os,re
import json


os.chdir('/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/vcf.germline')

####################
#function

#1. correct variant genomic position for indels
def GETREFALT(chro,ref,alt,pos,fa):
    if not ref or not alt:
        p = str(int(pos) - 1)
        AddBase = list(os.popen('samtools faidx '+fa+' '+chro+':'+p+'-'+p))[1].strip().upper()
        return AddBase+ref,AddBase+alt,p
    else:
        return ref,alt,pos

#2. return json object with tumor_DNA and germline_DNA count, pmid, dna_assay, etc...
def GENJSONOBJ(li,readL):
    JS = {'pmid':'26580448','vorigin':'germline','project':'pcgp'}
    if readL[0].split('.')[0].isdigit():
        JS['tumor_DNA_WGS_ref'] = int(readL[1].split('.')[0]) - int(readL[0].split('.')[0])
        JS['tumor_DNA_WGS_alt'] = readL[0].split('.')[0]
        JS['germline_DNA_WGS_ref'] = int(readL[3].split('.')[0]) - int(readL[2].split('.')[0])
        JS['germline_DNA_WGS_alt'] = readL[2].split('.')[0]
        JS['dna_assay'] = 'wgs'
    if readL[4].split('.')[0].isdigit():
        JS['tumor_DNA_WES_ref'] = int(readL[5].split('.')[0]) - int(readL[4].split('.')[0])
        JS['tumor_DNA_WES_alt'] = readL[4].split('.')[0]
        JS['germline_DNA_WES_ref'] = int(readL[7].split('.')[0]) - int(readL[6].split('.')[0])
        JS['germline_DNA_WES_alt'] = readL[6].split('.')[0]
        if 'dna_assay' not in JS:
            JS['dna_assay'] = 'wes'
    return JS




RefGenomeFa = '/research/rgs01/resgen/legacy/gb_customTracks/tp/genomes/hg19.gz'


#Get sample name info
#generate dictionary with donor name as key and sample name as value
#sample info extracted from two files 'pediatric.samples' and 'pediatric.samples.2'
SAM_Nam = {}
samNamfh = open('pediatric.samples')
for ps in samNamfh:
    psl = ps.strip().split('\t')
    if psl[1].startswith('SJ'):
        sNamL = psl[1].split('_')
        donor = sNamL[0]
        if donor in SAM_Nam:
            if '_D' in psl[1]:
                SAM_Nam[donor] = psl[1]
            else:
                continue
        else:
            SAM_Nam[donor] = psl[1]
    else:
        continue
samNamfh.close()

samNamfh2 = open('pediatric.samples.2')
for ps in samNamfh2:
    psl = ps.strip().split('\t')
    if psl[5].split('_')[0] in SAM_Nam:
        continue
    else:
        SAM_Nam[psl[5].split('_')[0]] = psl[5]
samNamfh2.close()

#screen 5 supplementary tables from NEJM paper
#SNVindelFile is a dictionary with supplementary table as key and the column of patient ID as value
#output germline variants in db store format
VAR = {}
RSID = {}
MissedSam = set()
SNVindelFile = {'GMua_60AutosomalDomPreGenes':2,'GMua_29AutosomalRecessiveGenes':2,'GMua_58SuppressorGenes':1,'GMua_23KinaseGenes':1,'GMua_395AddGenes':1}
for F in SNVindelFile.keys():
    fh = open(F)
    CaseCor = SNVindelFile[F]
    for line in fh:
        linel = line.split('\t')
        if not linel[0]:
            continue
        SamNamtem = linel[CaseCor]
        if SamNamtem == 'SJOS001108M1':
            SamNam = 'SJOS001108'
        elif SamNamtem.startswith('SJ'):
            SamNam = SamNamtem
        else:
            SamNam = re.search("(SJ.*)",SamNamtem).group(1)
        if SamNam in SAM_Nam:
            Sample = SAM_Nam[SamNam]
        else:
            Sample = SamNam
            MissedSam.add(SamNam)
        if linel[CaseCor+4]:
            rsID = linel[CaseCor+4]
        else:
            rsID = ''
        chron = linel[CaseCor+14]
        posTem = linel[CaseCor+15].split('.')[0]
        refTem = linel[CaseCor+16].strip().replace('-','')
        altTem = linel[CaseCor+17].strip().replace('-','')
        if not refTem and not altTem:
            continue
        ref,alt,pos = GETREFALT(chron,refTem,altTem,posTem,RefGenomeFa)
        CJS = GENJSONOBJ(linel,linel[CaseCor+21:CaseCor+28]+[linel[CaseCor+28].strip()])
        varID = '\t'.join([chron[3:],pos,ref,alt])
        if varID not in VAR:
            VAR[varID] = {Sample:CJS}
        else:
            VAR[varID][Sample] = CJS
        if rsID and varID not in RSID:
            RSID[varID] = rsID
    fh.close()

#Fill in tumor RNA allele count for germline mutation
#RNA allele count will be gathered from the json object RNACOUNT generated by allele count calculation pipeline
#tp/jwang/SNVCOUNTCALL/germline
#(1).generate snv file(chr,pos,ref,alt) for each sample. bcftools was used to do allele counting when RNAseq bam file is available
#(2).generate python json object(RNACOUNT) "8@71053502@C@T": {"SJE2A018_D": [35, 39]}

with open('RNACOUNT') as input:
    RNACOUNT = json.load(input)
    
for v in RNACOUNT:
    R2VID = '\t'.join(v.split('@'))
    if R2VID not in VAR:
        continue
    for sam in RNACOUNT[v]:
        if sam in VAR[R2VID]:
            VAR[R2VID][sam]['tumor_RNA_ref'] = int(RNACOUNT[v][sam][0])
            VAR[R2VID][sam]['tumor_RNA_alt'] = int(RNACOUNT[v][sam][1])


#output NEJMTRY.sqlite
out = open('NEJMGermline.sqlite','w')
for v in VAR:
    if v in RSID:
        rsid = RSID[v]
    else:
        rsid = '.'
    out.write('\t'.join([v,json.dumps(VAR[v],sort_keys=True),'.',rsid])+'\n')
out.close()

print('The file NEJMGermline.sqlite generated!')

The file NEJMTRY.sqlite generated!


In [25]:
#2. PCGP somatic mutations
#Collect somatic mutation from 22 PCGP papers, hg38 vcf from stjude cloud and proteinpaint sqlite db dataset
#The processing directory will be /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/mds.snvindel
#tumor_DNA_WGS, germline_DNA_WGS, tumor_DNA_WES, germline_DNA_WES, tumor_DNA_CC, germline_DNA_CC, tumor_RNA

import os
import subprocess as sp

os.chdir('/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/mds.snvindel')


#(1). Individual paper folder at hg19/PCGP/mds.snvindel
#Generate PCGP_PaperSNVindel.sqlite
#SNVs with same ref and alt are removed
#Germline SNVindels are removed
#Fix minus allele count
#Germline SNV misassigned as somatic 17	7579313	G	C      SJHYPO055_D	from 23334668
#Fix ref and alt alleles swap error ("24705251")

print('(1). PCGP individual paper...')

#(I). 2011_ETP_TALL 22237106
os.chdir('2011_ETP_TALL')
sp.run('python3 2011_ETP_TALL.py SNVindel_SuppTable 2011_ETP_TALL_SAMPLE',shell=True)
sp.run('cat 2011_ETP_TALL_SNVindel.sqlite >../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2011_ETP_TALL done')

#(II). 2014_EWS 25223734
os.chdir('../2014_EWS')
sp.run('python3 2014_EWS.py 2014_EWS_sample_IDswi SNVindel_SuppTable',shell=True)
sp.run('cat 2014_EWS_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2014_EWS done')

#(III). 2013_E_RHB 24332024
os.chdir('../2013_E_RHB')
sp.run('python3 2013_E_RHB.py SNV indel SNV_vali_cohort indel_vali_cohort',shell=True)
sp.run('cat 2013_E_RHB_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2013_E_RHB done')

#(IV). 2014_Ph-like_ALL 25207766
os.chdir('../2014_Ph-like_ALL')
sp.run('python3 2014_Ph-like_ALL.py 2014_Ph-like_ALL_snvindel_sum  SNVindel',shell=True)
sp.run('cat 2014_Ph-like_ALL_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2014_Ph-like_ALL done')

#(V). 2012_hypodiploid_ALL 23334668
os.chdir('../2012_hypodiploid_ALL')
sp.run('python3 2012_hypodiploid_ALL.py 2012_hypodiploid_ALL_SAMPLE SNVindel_WES',shell=True)
sp.run('cat 2012_hypodiploid_ALL_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2012_hypodiploid_ALL done')

#(VI). 2015_MEL 25268584
os.chdir('../2015_MEL')
sp.run('python3 2015_MEL.py SNVindel_WES SNVindel_WGS',shell=True)
sp.run('cat 2015_MEL_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2015_MEL done')

#(VII). 2014_HGG 24705251
os.chdir('../2014_HGG')
sp.run('python3 2014_HGG.py SNVindel_WES SNVindel_WGS',shell=True)
sp.run('cat 2014_HGG_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2014_HGG done')

#(VIII). 2012_MB 22722829
os.chdir('../2012_MB')
sp.run('python3 2012_MB.py SNVindel SNVindelT23',shell=True)
sp.run('cat 2012_MB_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2012_MB done')

#(IX). 2014_OS 24703847
os.chdir('../2014_OS')
sp.run('python3  2014_OS.py SAMPLE SNV',shell=True)
sp.run('cat 2014_OS_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2014_OS done')

#(X). 2014_ACT 25743702
os.chdir('../2014_ACT')
sp.run('python3 2014_ACT.py SNVs_s3a SNVs_s3b indel_s3c',shell=True)
sp.run('cat 2014_ACT_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2014_ACT done')

#(XI). 2013_LGG 23583981
#remove SJLGG022_D      17      29664899 G A (germline)
os.chdir('../2013_LGG')
sp.run('python3 2013_LGG.py SNVindel_39WGS',shell=True)
sp.run('cat 2013_LGG_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2013_LGG done')

#(XII). 2012_AMLM7 23153540
#no chromosome info for SNVindel 
#consider to add sample table

#(XIII). 2014_EPD 24553141
os.chdir('../2014_EPD')
sp.run('python3 2014_EPD.py SAMPLE_match SNVindel_T123',shell=True)
sp.run('cat 2014_EPD_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2014_EPD done')

#(XIV). 2014_INF 25730765
os.chdir('../2014_INF')
sp.run('python3 2014_INF.py S36SNVsMLL S2SNVs',shell=True)
sp.run('cat 2014_INF_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2014_INF done')

#(XV). 2014_RB 24509483
#10 WGS, no SNVindel 

#(XVI). 2012_RB 22237022
#4 Diagnosis samples, SNVindels for X not included
os.chdir('../2012_RB')
sp.run('python3 2012_RB.py SNVindel',shell=True)
sp.run('cat 2012_RB_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2012_RB done')

#(XVII). 2012_NBL
#no variation report
#consider to add sample table

#(XVIII). 2015_CPC
#consider to add sample table

#(XIX). 2013_1000paediatric
#no readcount, ref, alt info for SNVindel

#(XX). 2014_pediatric_BALL 25790293
#condider to add CNV (snp6)
os.chdir('../2014_pediatric_BALL')
sp.run('python3 2014_pediatric_BALL.py sampleMatch snvindel',shell=True)
sp.run('cat 2014_pediatric_BALL_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2014_pediatric_BALL done')

#(XXI). 2016_ALL 
os.chdir('../2016_ALL')
sp.run('python3 2016_ALL.py SAMPLE SNVindels',shell=True)
sp.run('cat 2016_ALL_SNVindel.sqlite >>../PCGP_PaperSNVindel.sqlite',shell=True)
print('\t2016_ALL done')

#(XXII). 2016_AML

os.chdir('../')
print('PCGP individual paper done')







#(2). hg38 vcf from stjude cloud to hg19 data store format 
#Generate PCGP652_SNVindel.sqlite
#Only use samples that are associated with DNA assay and PMID
#Fix minus allele count /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/vcf.somatic/hg38-CountFix.py

print('(2). hg38 VCF, coding & noncoding...')

#converst hg38 vcf to data store
sp.run('python3 ~/tp/utils/VCF2SQLite.py -v /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/vcf.somatic/PCGP652_SNVindel.vcf -o PCGP652_SNVindel.sqlite --rsid',shell=True)
print('\tPCGP652_SNVindel.sqlite generated!')

#Add attributes dna_assay etc...
sp.run('python3 hg38AddAttr.py PCGP_SAMPLES PCGP652_SNVindel.sqlite',shell=True)
sp.run('rm -f PCGP652_SNVindel.sqlite',shell=True)
sp.run('mv PCGP652_SNVindel.sqlite.new PCGP652_SNVindel.sqlite',shell=True)
print('\tAttributes added!')

#remove tumor_RNA count
sp.run('python3 SQLiteRNAC_RM.py PCGP652_SNVindel.sqlite',shell=True)
sp.run('rm -f PCGP652_SNVindel.sqlite',shell=True)
sp.run('mv PCGP652_SNVindel.sqlite_new PCGP652_SNVindel.sqlite',shell=True)
print('\ttumor_RNA count removed from data store!')

print('hg38 VCF, coding & noncoding done!')






#(3). legacy SNV/indel table from sqlite db tp/anno/db/pecan-prod/snv_indel.tsv
#Generate pediatric.hg19.db.sqlite
#Only use samples that are associated with DNA assay and PMID
#Correct read count for SNVs from WGS and have two lines from sql db db2SQLite.py (line:153) "db2SQLite.py"
#Germline SNV misassigned as somatic 2    223163303    A    C    SJRHB046_D "db2SQLite.py"

print('(3). legacy SNV/indel table from sqlite db...')

#convert sqlite 3 db to SNVindel data store
sp.run('python3 db2SQLite.py PCGP_SAMPLES /research/rgs01/resgen/legacy/gb_customTracks/tp/anno/db/pecan-prod/snv_indel.tsv pediatric.hg19.db.sqlite',shell=True)
print('\tconvert sqlite 3 db to SNVindel data store. pediatric.hg19.db.sqlite generated!')

#remove tumor_RNA count
sp.run('python3 SQLiteRNAC_RM.py pediatric.hg19.db.sqlite',shell=True)
sp.run('rm -f pediatric.hg19.db.sqlite',shell=True)
sp.run('mv pediatric.hg19.db.sqlite_new pediatric.hg19.db.sqlite',shell=True)
print('\ttumor_RNA count removed from data store!')

print('legacy SNV/indel table from sqlite db is converted to data store!')







#(4). Merge data store files from 3 difference sources
# PCGP_PaperSNVindel.sqlite PCGP652_SNVindel.sqlite pediatric.hg19.db.sqlite to PCGP.sqlite
# dna_assay type preference order for a specific sample from different source during merge process:‘wgs’ > ‘wes’ > ‘cc’
print('(4). Merging data store file...')
sp.run('python3 sqliteMerge.py --SQLpaper PCGP_PaperSNVindel.sqlite --SQLhg38 PCGP652_SNVindel.sqlite --SQLSQL pediatric.hg19.db.sqlite -o PCGP.sqlite',shell=True)
print('Merged! PCGP.sqlite generated.')






#(5). Fill in tumor RNA allele count for PCGP mutation
# Precalculated PCGP RNA allele count
# RNA allele count will be gathered from the json object PCGPSNVRNACOUNT generated by allele count calculation pipeline
# hg19/PCGP/mds.snvindel/DNARNACOUNT/PCGPSNVRNACOUNT
# How to generate PCGPSNVRNACOUNT
# (1). generate snv file(chr,pos,ref,alt) from high20 file for each sample (only inlude somatic snv). snvGenerator.py -> hg19/PCGP/mds.snvindel/DNARNACOUNT/runHigh20ToSNV.py
# (2). generate vcf file from snv bcftools was used to do allele counting when RNAseq bam file is available hg19/PCGP/mds.snvindel/DNARNACOUNT/snv2vcf.py
# (3). generate python json object(PCGPSNVRNACOUNT) "[RNASEQSAM,RNACOUNT{8@71053502@C@T": {"SJE2A018_D": [35, 39])}}]"
print('(5). Add RNA count for PCGP samples...')
sp.run('python3 AddRNACount.py -r ./DNARNACOUNT/PCGPSNVRNACOUNT -s PCGP.sqlite',shell=True)
sp.run('rm -f PCGP.sqlite',shell=True)
print('Done!\nPCGP.sqlite.RNACOUNT generated.')




#(6). Fill in PCGP Germline allele count
# Germline allele count will be gathered from WGS or WES high20 file
# hg19/PCGP/mds.snvindel/DNARNACOUNT/GERMCOUNT
#How to generate GERMCOUNT
# (1). generate snv file(chr,pos,ref,alt) from high20 file for each sample (only inlude somatic snv). snvGenerator.py -> hg19/PCGP/mds.snvindel/DNARNACOUNT/runHigh20ToSNV.py
# (2). generate python json object(GERMCOUNT) (last two column of snv file) "{"WES": {"7@107732066@C@T": {"SJAMLM7014_D": [104, 1]}...,"WGS":...}
print('(6). Add PCGP Germline allele count...')
sp.run('python3 AddGermCount.py -g ./DNARNACOUNT/GERMCOUNT -s PCGP.sqlite.RNACOUNT',shell=True)
print('Done!\nPCGP.sqlite.RNACOUNT.GERMDNACOUNT generated.')







(1). PCGP individual paper...
	2011_ETP_TALL done
	2014_EWS done
	2013_E_RHB done
	2014_Ph-like_ALL done
	2012_hypodiploid_ALL done
	2015_MEL done
	2014_HGG done
	2012_MB done
	2014_OS done
	2014_ACT done
	2013_LGG done
	2014_EPD done
	2014_INF done
	2012_RB done
	2014_pediatric_BALL done
	2016_ALL done
PCGP individual paper done
(2). hg38 VCF, coding & noncoding...
	PCGP652_SNVindel.sqlite generated!
	Attributes added!
	tumor_RNA count removed from data store!
hg38 VCF, coding & noncoding done!
(3). legacy SNV/indel table from sqlite db...
	convert sqlite 3 db to SNVindel data store. pediatric.hg19.db.sqlite generated!
	tumor_RNA count removed from data store!
legacy SNV/indel table from sqlite db is converted to data store!
(4). Merging data store file...
Merged! PCGP.sqlite generated.
(5). Add RNA count for PCGP samples...
Done!
PCGP.sqlite.RNACOUNT generated.
(6). Add PCGP Germline allele count...
Done!
PCGP.sqlite.RNACOUNT.GERMDNACOUNT generated.


In [27]:
#3. TARGET somatic mutations
#Collect somatic mutation from TARGET coding and noncoding tables
#The processing directory will be /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/TARGET/mds.snvindel
#tumor_DNA_CGI/WGS/WES, germline_DNA_CGI/WGS/WES, tumor_RNA
#pmid: 29489755, project: pantarget, vorigin: somatic

import os
import subprocess as sp

os.chdir('/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/TARGET/mds.snvindel')

# (1). coding
# Fix minus allele count X	133511749	T	TACACTGCCCGGGA
# Fill in TAL1 promoter insertion hg19/TARGET/mds.snvindel/TAL1

print('(1). panTARGET coding variants...')
sp.run('python3 table2datastore_coding.py -s D6SJIDTARGETdiagnosis -i snvindel_target -o pantarget_code.sqlite --TAL1 TAL1',shell=True)
print('Done! pantarget_code.sqlite generated.')

# (2). noncoding
# Collect noncoding mutation from 6 tables
#/research/rgs01/project_space/zhanggrp/PanTARGET/common/yanling/pan_target/CGI/annovar_rescue_filterStep3/header_line
#/research/rgs01/project_space/zhanggrp/PanTARGET/common/yanling/pan_target/CGI/annovar_rescue_filterStep3/deliver.all_tier23
#/research/rgs01/project_space/zhanggrp/PanTARGET/common/yanling/pan_target/CGI/annovar_rescue_filterStep3/deliver.aml_tier23
#/research/rgs01/project_space/zhanggrp/PanTARGET/common/yanling/pan_target/CGI/annovar_rescue_filterStep3/deliver.nbl_tier23
#/research/rgs01/project_space/zhanggrp/PanTARGET/common/yanling/pan_target/CGI/annovar_rescue_filterStep3/deliver.os_tier23
#/research/rgs01/project_space/zhanggrp/PanTARGET/common/yanling/pan_target/CGI/annovar_rescue_filterStep3/deliver.wt_tier23
print('(2). panTARGET noncoding variants...')
sp.run('python3 table2datastore_noncoding.py -c DIAGCGID --sampleinfo D6SJIDTARGETdiagnosis -o pantarget_noncoding.sqlite /research/rgs01/project_space/zhanggrp/PanTARGET/common/yanling/pan_target/CGI/annovar_rescue_filterStep3/header_line /research/rgs01/project_space/zhanggrp/PanTARGET/common/yanling/pan_target/CGI/annovar_rescue_filterStep3/deliver.all_tier23 /research/rgs01/project_space/zhanggrp/PanTARGET/common/yanling/pan_target/CGI/annovar_rescue_filterStep3/deliver.aml_tier23 /research/rgs01/project_space/zhanggrp/PanTARGET/common/yanling/pan_target/CGI/annovar_rescue_filterStep3/deliver.nbl_tier23 /research/rgs01/project_space/zhanggrp/PanTARGET/common/yanling/pan_target/CGI/annovar_rescue_filterStep3/deliver.os_tier23 /research/rgs01/project_space/zhanggrp/PanTARGET/common/yanling/pan_target/CGI/annovar_rescue_filterStep3/deliver.wt_tier23',shell=True)
print('Done! pantarget_noncoding.sqlite generated.')

# (3). Merging coding and noncoding
print('(3). Merging coding and noncoding...')
!python3 CodeNoncode_Merge.py -c pantarget_code.sqlite -n pantarget_noncoding.sqlite -o target.sqlite
print('Done! target.sqlite generated.')

# (4). Add panTarget signature
# Two files are used to add signature info: target_signature (SNV-signature probability matrix) and list of samples and valid signatures for each sample (tp/hg19/TARGET/mds.snvindel/pantarget.Signature.samples)
print('(3). Adding panTarget signature...')
!python3 datastore_addSignature.py -d target.sqlite -s target_signature --panSigSample pantarget.Signature.samples --sigStregth panSignatureStrength.new -g /research/rgs01/resgen/legacy/gb_customTracks/tp/genomes/hg19.gz -o target.signature.sqlite


# (5). Remove tumor_RNA count
print('(4). Remove tumor RNA count...')
!python3 SQLiteRNAC_RM.py target.signature.sqlite
!rm -f target.signature.sqlite
!mv target.signature.sqlite_new target.signature.sqlite

# (6). Add RNA count
# Precalculated PCGP RNA allele count
# RNA allele count will be gathered from the json object TARGET_MDS_RNAC generated by allele count calculation pipeline
# tp/hg19/TARGET/DNA/snp.ase/rna/TARGET_MDS_RNAC
# How to generate PCGPSNVRNACOUNT
# (1). generate snv file(chr,pos,ref,alt) from high20(WES) or MAF file (CGI) file for each sample. tp/hg19/TARGET/DNA/snp.ase/cgi/do.sh and tp/hg19/TARGET/DNA/snp.ase/wes/do.sh
# (2). generate vcf file from snv bcftools was used to do allele counting when RNAseq bam file is available tp/hg19/TARGET/DNA/snp.ase/rna/Allele_Count.py
# (3). generate python json object including all high20 snv vcf2AlleleCount.py
# (4). generate python json object including only mds snv mds_RNACOUNT.py 
print('(5). Add RNA count...')
!python3 AddRNACount.py -r /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/TARGET/DNA/snp.ase/rna/TARGET_MDS_RNAC -s target.signature.sqlite
!rm -f target.signature.sqlite
print('Done! target.signature.sqlite.RNACOUNT generated.')



(1). panTARGET coding variants...
Done! pantarget_code.sqlite generated.
(2). panTARGET noncoding variants...
Done! pantarget_noncoding.sqlite generated.
(3). Merging coding and noncoding...
Done! target.sqlite generated.
(3). Adding panTarget signature...
(4). Remove tumor RNA count...
(5). Add RNA count...
840
Done! target.signature.sqlite.RNACOUNT generated.


In [ ]:
#SCMC
#Add 13 SCMC TALL 
#'SJALL018373_D1', 'SJALL018390_D1', 'SJALL040466_D1', 'SJALL040467_D1', 'SJALL040468_D1', 'SJALL040472_D1', 'SJALL043558_D1', 'SJALL043857_D1', 'SJALL043861_D1', 'SJALL043869_D1', 'SJTALL013797_D1', 'SJTALL013800_D1', 'SJTALL013801_D1'
#pmid: 31697823
#generate /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/SCMC/2020_SCMC/SCMC.snvindel.sqlite
#os.chdir('/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/SCMC/2020_SCMC/')
#sp.run('python3 2020_SCMC.py SCMC.snvindel',shell=True)
#print('\t2020_SCMC done')

In [2]:
#4. Combine SNVindels from NEJM germline, PCGP and TARGET and Xenograft
# /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/vcf.germline/NEJMGermline.sqlite
# /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/mds.snvindel/PCGP.sqlite.RNACOUNT.GERMDNACOUNT
# /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/TARGET/mds.snvindel/target.signature.sqlite.RNACOUNT
#/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/SCMC/2020_SCMC/SCMC.snvindel.sqlite
#/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/SCMC/2020_SCMC/cis-X.indel.sqlite
#/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/Pediatric/Pediatric_SNVindel_vcf_generator/panNBL_33056981/pannbl.snvindel.sqlite
#./Xenograft_SNVindel.vep.sqlite 
# Xenograft_SNVindel SJNBL046_X, SJOS001112_X1, SJRHB000026_X1, SJRHB010463_X16
# script used for combination tp/utils/SQLitesMerge.py

import os

os.chdir('/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/Pediatric/Pediatric_SNVindel_vcf_generator')
!python3 /research/rgs01/resgen/legacy/gb_customTracks/tp/utils/SQLitesMerge.py -o pediatric.hg19.sqlite --anno --rsid --SQLites /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/vcf.germline/NEJMGermline.sqlite /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/mds.snvindel/PCGP.sqlite.RNACOUNT.GERMDNACOUNT /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/TARGET/mds.snvindel/target.signature.sqlite.RNACOUNT /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/SCMC/2020_SCMC/SCMC.snvindel.sqlite /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/SCMC/2020_SCMC/cis-X.indel.sqlite /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/Pediatric/Pediatric_SNVindel_vcf_generator/panNBL_33056981/pannbl.snvindel.sqlite Xenograft_SNVindel.vep.sqlite 
print('pediatric.hg19.sqlite generated!')


pediatric.hg19.sqlite generated!


In [3]:
#5. data store to vcf file
# vcf header file should be pre-prepared (VCFHeader)

import os,sys

if not os.path.isfile('VCFHeader'):
    print('Error: VCF header file doesn\'t exist!',file=sys.stderr)
    print('Folder: '+os.getcwd())
    sys.exit(1)
    
!python3 /research/rgs01/resgen/legacy/gb_customTracks/tp/utils/SQLite2VCF.py -j pediatric.hg19.sqlite --header VCFHeader -o pediatric.hg19.vcf --anno --rsid
print('pediatric.hg19.vcf generated!')

3765
pediatric.hg19.vcf generated!


In [1]:
#6. VEP annotation
# VEP was locally installed and VEP dataset can be found from /home/jwang7/.vep
# For different user, please modify vep command line accordingly
# Run next cell/step when VEP annotation is done and pediatric.hg19.vep.vcf is generated

import subprocess as sp
import os

if os.path.isfile('VCF_log'):
    sp.run('rm -f VCF_log',shell=True)

out = open('RUNVEP.sh','w')
out.write('#!/bin/bash\n#BSUB -J VCF\n#BSUB -q gpu_rhel7\n')
out.write('#BSUB -n 20\n#BSUB -R "rusage[mem=4000]"\n#BSUB -o VCF_log\n#BSUB -e VCF_elog\n')
out.write('vep --cache --cache_version 100 --assembly GRCh37 --refseq --dir /research/rgs01/resgen/legacy/gb_customTracks/tp/jwang/tools/VEP/.vep/ --fork 20 --offline --hgvs --fasta /research/rgs01/resgen/legacy/gb_customTracks/tp/jwang/tools/VEP/Homo_sapiens.GRCh37.75.dna.toplevel.fa.gz -i pediatric.hg19.vcf --vcf -o pediatric.hg19.vep.vcf')
out.close()

#submit job
sp.run('bsub <RUNVEP.sh',shell=True)

CompletedProcess(args='bsub <RUNVEP.sh', returncode=0)

In [3]:
#7. sort, bgzip vcf file

import os,sys 
import subprocess as sp

if not os.path.isfile('VCF_log'):
    print('VEP annotation not finished yet!',file=sys.stderr)
    sys.exit(1)
    
sp.run('mv pediatric.hg19.vep.vcf pediatric.hg19.vcf',shell=True)
!bcftools sort -T ./ -o pediatric.hg19.vcf.sorted pediatric.hg19.vcf
sp.run('mv pediatric.hg19.vcf.sorted pediatric.hg19.vcf',shell=True)
!bgzip pediatric.hg19.vcf
!tabix -p vcf pediatric.hg19.vcf.gz
print('pediatric.hg19.vcf.gz and pediatric.hg19.vcf.gz.tbi generated')



Writing to ./
Merging 148 temporary files
Cleaning
Done
pediatric.hg19.vcf.gz and pediatric.hg19.vcf.gz.tbi generated


In [43]:
"""
# Adding more SNVindel mutation
# 01. Xenograft_SNVindel SJNBL046_X, SJOS001112_X1, SJRHB000026_X1, SJRHB010463_X16
# SNVindel data has to be stored in the format of data store with vep annotation Xenograft_SNVindel.vep.sqlite

import os
import subprocess as sp


#copy current pediatric.hg19.vcf.gz to current folder
#add new data
sp.run('cp ../pediatric.hg19.vcf.gz ./',shell=True)
!python3 /research/rgs01/resgen/legacy/gb_customTracks/tp/utils/VCF2SQLite.py -v pediatric.hg19.vcf.gz -o current.sqlite --anno --rsid
!python3 /research/rgs01/resgen/legacy/gb_customTracks/tp/utils/SQLitesMerge.py -o tem.full.sqlite --anno --rsid --SQLites current.sqlite Xenograft_SNVindel.vep.sqlite
sp.run('rm -f current.sqlite pediatric.hg19.vcf.gz',shell=True)
print('new data added!')

#convert data store to vcf
!python3 /research/rgs01/resgen/legacy/gb_customTracks/tp/utils/SQLite2VCF.py -j tem.full.sqlite --header VCFHeader -o pediatric.hg19.vcf --anno --rsid
#sp.run('rm -f tem.full.sqlite',shell=True)

#sort, bgzip and index vcf file
!bcftools sort -T ./ -o pediatric.hg19.vcf.sorted pediatric.hg19.vcf
sp.run('mv pediatric.hg19.vcf.sorted pediatric.hg19.vcf',shell=True)
!bgzip pediatric.hg19.vcf
!tabix -p vcf pediatric.hg19.vcf.gz

print('Vcf file update done')
"""

new data added!
3207
Writing to ./
Merging 124 temporary files
Cleaning
Done
Done


In [ ]:
# VCF split by sample and mutation by sample
# This step should be always the last step (last cell) in this notebook
# This step should be done after update of vcf file


# vcf split by sample
print('vcf split by sample...')

# mutation by sample
print('mutation by sample...')